In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split,cross_val_score
import sklearn.metrics as mt
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV

In [ ]:
data = pd.read_csv("USA_Housing.csv")
df = data.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(columns="Address",axis=1)
df

In [ ]:
df.isnull().sum()

In [ ]:
#Bağımsız değişkenler arasında bir ilişki olmamasını kontrol etmek için grafik çiziyoruz
sns.pairplot(df)
plt.show()

In [ ]:
#Bağımsız değişkenler arasında bir ilişki olmamasını kontrol etmek için korelasyon yapısına bakıyoruz
sns.heatmap(df.corr(),annot=True)
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Bağımsız değişkenler arasında bir ilişki olmamasını kontrol etmek için VIF yapısına bakıyoruz
y = df["Price"]
X = df.drop("Price",axis=1)

sabit = sm.add_constant(X)

vif = pd.DataFrame()
vif["Değişkenler"] = X.columns
vif["VIF"] = [variance_inflation_factor(sabit,i+1) for i in range(X.shape[1])]
vif # 5 değerinin üzerindekiler sorun, altındakiler sorun değil

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#cross_val_score = İlk parametresi hangi model olduğu | 2. bağımsız | 3. Bağımlı | 4. Kaç kat parçalayacaksın
def caprazdog(model):
    dogruluk = cross_val_score(model,X,y,cv=10)
    return dogruluk.mean()
    #Her bir parçalama için bir skor ortaya çıkarır ve bu skorların ortalamasını bize döndürür
def basari(gercek,tahmin):
    mse = mt.mean_squared_error(gercek,tahmin,squared=True)
    r2 = mt.r2_score(gercek,tahmin)
    return [mse,r2]
lin_model = LinearRegression()
lin_model.fit(X_train,y_train)
lin_tahmin = lin_model.predict(X_test)

ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X_train,y_train)
ridge_tahmin = ridge_model.predict(X_test)

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train,y_train)
lasso_tahmin = lasso_model.predict(X_test)

elas_model = ElasticNet(alpha=0.1)
elas_model.fit(X_train,y_train)
elas_tahmin = elas_model.predict(X_test)

sonuclar = [["Linear Model",basari(y_test,lin_tahmin)[0],basari(y_test,lin_tahmin)[1],caprazdog(lin_model)],
           ["Ridge Model",basari(y_test,ridge_tahmin)[0],basari(y_test,ridge_tahmin)[1],caprazdog(ridge_model)],
           ["Lasso Model",basari(y_test,lasso_tahmin)[0],basari(y_test,lasso_tahmin)[1],caprazdog(lasso_model)],
           ["ElasticNet Model",basari(y_test,elas_tahmin)[0],basari(y_test,elas_tahmin)[1],caprazdog(elas_model)],
           ]
 
pd.options.display.float_format = '{:,.4f}'.format # Tüm veri çerçevesinde bulunan MSE sütunundaki bilimsel sayıyı
#virgülden sonra 4 değeri ile gösteriyoruz
sonuclar = pd.DataFrame(sonuclar,columns=["Model","MSE","R2","Doğrulama"])
sonuclar #MSE değeri düşük olmalıdır
#R2 değeri yüksek olmalıdır
#Doğrulama yüksek olmalıdır

In [ ]:
#Alpha değerlerini optimal hale getirme işlemi
#Ridge için
lambdalar = 10**np.linspace(10,-2,100)*0.5
ridge_lamb = RidgeCV(alphas=lambdalar, scoring="r2").fit(X_train,y_train).alpha_
ridge_model = Ridge(alpha=ridge_lamb)
ridge_model.fit(X_train,y_train)
ridge_tahmin = ridge_model.predict(X_test)


#Lesso için
lasso_lamb = LassoCV(cv=10,max_iter=10000).fit(X_train,y_train).alpha_
lasso_model = Lasso(alpha=lasso_lamb)
lasso_model.fit(X_train,y_train)
lasso_tahmin = lasso_model.predict(X_test)


#ElasticNet için
elastic_lamb = ElasticNetCV(cv=10,max_iter=10000).fit(X_train,y_train).alpha_
elas_model = ElasticNet(alpha=elastic_lamb)
elas_model.fit(X_train,y_train)
elas_tahmin = elas_model.predict(X_test)


sonuclar = [["Linear Model",basari(y_test,lin_tahmin)[0],basari(y_test,lin_tahmin)[1],caprazdog(lin_model)],
           ["Ridge Model",basari(y_test,ridge_tahmin)[0],basari(y_test,ridge_tahmin)[1],caprazdog(ridge_model)],
           ["Lasso Model",basari(y_test,lasso_tahmin)[0],basari(y_test,lasso_tahmin)[1],caprazdog(lasso_model)],
           ["ElasticNet Model",basari(y_test,elas_tahmin)[0],basari(y_test,elas_tahmin)[1],caprazdog(elas_model)],
           ]

pd.options.display.float_format = '{:,.4f}'.format
sonuclar = pd.DataFrame(sonuclar,columns=["Model","MSE","R2","Doğrulama"])
sonuclar

